# Data Owner

In [ ]:
from AriesExchanger import AriesDuetTokenExchanger
%autoawait
import time
import asyncio
import nest_asyncio
from aries_basic_controller.aries_controller import AriesAgentController

## Instantiate the Aries Controller

Note the current arguments assume you are running the docker-compose environment locally. You will need to change this if you are running it on a VM.

In [ ]:
# Anyone know how to get env variables into juypter notebooks through docker easily?
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8052
WEBHOOK_BASE = ""
ADMIN_URL = "http://0.0.0.0:8051"

# Note AttachmentController just extends the basic controller to include the new attachment endpoint
dataowner_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL)

In [ ]:
dataowner_exchanger = AriesDuetTokenExchanger(agent_controller=dataowner_controller)

## Get Data Owner Credential from Open Mined Authority

You will need to navigate to the OM Authority application, authenticate by creating a connection using the mobile wallet that contains the OM PKI credential from the previous notebooks and the create a Data Owner invitation and copy this across.

In [ ]:
invite = {"@type":"did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation","@id":"603adb0f-4716-4c8d-8988-939c61c0aaf1","label":"OM_AUTHORITY","recipientKeys":["6GML86XMsJMsBJdku6yJjZRxoE3UkAFkKxY7N72FJH6U"],"serviceEndpoint":"https://0a0f5987e371.ngrok.io"}

In [ ]:
dataowner_exchanger.accept_connection(invite)

## Configure Duet Server Challenge

This is a proof request that is made to every entity who attempts to make a connection with the duet server. Before receiving the relevant token they must respond with a valid presentation to this request.

Note this challenge could be anything you want, designed to establish context specific trust dependant on the use case. For the challenge to be useful you must be confident that the correct entities will be able to get access to the credential needed to respond. 

The Data Owner is going to challenge all connections to present the scope attribute issued as part of the following schema `Sgg1wREgfEwbEPCQn9xEuE:2:OM Data Scientist:0.0.1`. 

Note: An extension might include how to use the value of the scope attribute presented to define permissions.

In [ ]:
schema_id =  "Sgg1wREgfEwbEPCQn9xEuE:2:OM Data Scientist:0.0.1"


revocation = False
exchange_tracing = False

#Enable this to ask for attributes to identity a user
#TODO - change restriction to schemaId or credentialId

req_attrs = [
    {"name": "scope", "restrictions": [{"schema_id": schema_id}]},
]

global indy_proof_request
indy_proof_request = {
    "name": "Proof of Scientist",
    "version": "1.0",
    "requested_attributes": {
        f"0_{req_attr['name']}_uuid":
        req_attr for req_attr in req_attrs
    },
    "requested_predicates": {
#         f"0_{req_pred['name']}_GE_uuid":
#         req_pred for req_pred in req_preds
    },
}

In [ ]:
dataowner_exchanger.configure_challenge(indy_proof_request)

## Launch Duet Session

Notice how we are passing in an optional token_exchanger. This must be a DuetTokenExchanger, that implements a mechanisms to send and recieve a duet token identifier with the other PPML participant. In this case the Data Scientist.

As we are using the AriesExchanger class, this is done across a DIDComm connection that first must be established. So rather than printing the duet token and asking you to manually copy this across, we print out an aries invitation. 

In [ ]:
import syft as sy
duet = sy.launch_duet(credential_exchanger=dataowner_exchanger)

In [ ]:
duet